In [1]:
#import relevant packages for analysis

from __future__ import print_function

import os
import sys

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

PROJ_ROOT = os.path.join(os.pardir)

In [2]:
terry_data = os.path.join(PROJ_ROOT, 
                         'data',
                         'Terry_Stops.csv')

df = pd.read_csv(terry_data)

In [3]:
df.head(2)

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,18 - 25,NaN,20150000202641,11662,-,None,1128,1971.0,M,Hispanic or Latino,...,18:23:00.0000000,ROBBERY - IP/JO (INCLUDES STRONG ARM),--ROBBERY - STRONG ARM,911,SOUTH PCT 1ST W - ROBERT,N,N,South,R,R3
1,46 - 55,NaN,20150000224628,12260,-,None,1690,1980.0,M,White,...,19:39:00.0000000,ASLT - IP/JO - PERSON SHOT OR SHOT AT,--HOMICIDE,911,EAST PCT 2ND W - BEATS,N,N,East,C,C3


I will not need all of these columns for my analysis. I will proceed to inspect the columns and clean the columns that will be kept

In [4]:
df.shape

(34521, 23)

In [5]:
df.columns

Index(['Subject Age Group', 'Subject ID', 'GO / SC Num', 'Terry Stop ID',
       'Stop Resolution', 'Weapon Type', 'Officer ID', 'Officer YOB',
       'Officer Gender', 'Officer Race', 'Subject Perceived Race',
       'Subject Perceived Gender', 'Reported Date', 'Reported Time',
       'Initial Call Type', 'Final Call Type', 'Call Type', 'Officer Squad',
       'Arrest Flag', 'Frisk Flag', 'Precinct', 'Sector', 'Beat'],
      dtype='object')

In [6]:
# create a sub dataframe with the selected columns needed for analysis
df = df[['Subject Age Group', 'Subject ID', 'Stop Resolution',
         'Weapon Type', 'Officer ID', 'Officer YOB', 
         'Officer Gender', 'Officer Race', 'Subject Perceived Race', 
         'Subject Perceived Gender', 'Reported Date', 'Initial Call Type','Call Type',
         'Officer Squad', 'Arrest Flag', 'Frisk Flag',
         'Precinct', 'Sector', 'Beat' ]]


In [7]:
# rename the columns by remvoing the space and capital letters

df.columns = ['subject_age', 'subject_id', 'stop_resolution',
         'weapon_type', 'officer_id', 'officer_age', 
         'officer_gender', 'officer_race', 'subject_race', 
         'subject_gender', 'date', 'initial_call_type', 'call_type',
         'officer_squad', 'arrest', 'frisk',
         'precinct', 'sector', 'beat' ]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34521 entries, 0 to 34520
Data columns (total 19 columns):
subject_age          34521 non-null object
subject_id           33285 non-null float64
stop_resolution      34521 non-null object
weapon_type          34521 non-null object
officer_id           34521 non-null int64
officer_age          34519 non-null float64
officer_gender       34521 non-null object
officer_race         34521 non-null object
subject_race         34521 non-null object
subject_gender       34521 non-null object
date                 34521 non-null object
initial_call_type    21968 non-null object
call_type            21968 non-null object
officer_squad        34153 non-null object
arrest               34521 non-null object
frisk                34521 non-null object
precinct             21968 non-null object
sector               21968 non-null object
beat                 21968 non-null object
dtypes: float64(2), int64(1), object(16)
memory usage: 5.0+ MB


In [9]:
# change the data type for the category & datetime columns
df = df.astype({'subject_age': 'category', 'officer_gender':'category', 
                'officer_race': 'category', 'subject_race':'category', 
                'subject_gender':'category', 'date':'datetime64','arrest':'category', 
                'frisk':'category','precinct':'category', 'sector':'category',
                'beat': 'category'})

In [10]:
#replace '-' with NaN values
df.replace('-', np.nan, inplace = True)

## Explore the value of the columns

In [11]:
# There are multiple NaN values for the subjects age - these will be left in, as these are originally perceived age groups 
# from officers, and will not affect a portion of the analysis
df.subject_age.value_counts(dropna = False)

26 - 35         11497
18 - 25          7457
36 - 45          6985
46 - 55          4324
1 - 17           1670
56 and Above     1573
NaN              1015
-                   0
Name: subject_age, dtype: int64

In [12]:
df.officer_age.value_counts(dropna = False, ascending = True).head(3)

1951.0    1
NaN       2
1946.0    2
Name: officer_age, dtype: int64

In [13]:
#remove the 2 NaNs from the officer_age column by filling them in with the mean age
df.officer_age.fillna(int(df.officer_age.mean()), inplace = True)

In [14]:
#replace the original 'officer YOB' column with the age of the officer, using the year of the report & the officer YOB
df['officer_age'] = (df.date.dt.year - df.officer_age)

In [15]:
df.officer_age.describe()

count    34521.000000
mean        35.137945
std          8.625263
min         21.000000
25%         29.000000
50%         33.000000
75%         40.000000
max        118.000000
Name: officer_age, dtype: float64

In [16]:
# view the top range of the age range for officers
df[df.officer_age >55]

,subject_age,subject_id,stop_resolution,weapon_type,officer_id,officer_age,officer_gender,officer_race,subject_race,subject_gender,date,initial_call_type,call_type,officer_squad,arrest,frisk,precinct,sector,beat
8,NaN,NaN,NaN,None,188,62.0,M,Asian,NaN,NaN,2015-08-05,UNKNOWN - ANI/ALI - LANDLINE (INCLUDES OPEN LINE),911,NORTH PCT 2ND W - LINCOLN,N,NaN,North,L,L1
18,NaN,NaN,NaN,None,289,58.0,M,White,NaN,NaN,2016-05-09,NaN,NaN,SOUTHWEST PCT 2ND WATCH - F/W RELIEF,N,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,None,447,58.0,M,White,NaN,NaN,2016-02-21,NaN,NaN,SOUTHWEST PCT 2ND W - WILLIAM,N,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,None,607,57.0,M,White,Unknown,Unable to Determine,2018-05-11,TRAFFIC STOP - OFFICER INITIATED ONVIEW,ONVIEW,NaN,N,N,West,K,K3
98,36 - 45,5401.0,NaN,None,904,57.0,F,White,White,Male,2016-10-01,NaN,NaN,SOUTHWEST PCT - 1ST WATCH - F/W RELIEF,N,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34380,26 - 35,21129.0,Street Check,None,1076,56.0,F,White,White,Male,2017-09-11,NaN,NaN,WEST PCT 2ND W - DAVID,N,N,NaN,NaN,NaN
34387,46 - 55,21358.0,Street Check,None,505,58.0,M,White,Black,Male,2018-10-29,NaN,NaN,WEST PCT 2ND W - MARY BEATS,N,N,NaN,NaN,NaN
34400,36 - 45,21615.0,Street Check,None,447,57.0,M,White,White,Male,2015-12-19,NaN,NaN,WEST PCT 2ND W - DAVID BEATS,N,N,NaN,NaN,NaN
34456,36 - 45,23358.0,Street Check,None,764,57.0,M,White,White,Male,2016-08-12,NaN,NaN,WEST PCT 2ND W - QUEEN,N,N,NaN,NaN,NaN


In [17]:
df = df.drop(df[df.officer_age >55].index)

In [18]:
#No additional negative values similar to above
df.officer_id.describe()

count    33683.000000
mean      1880.765371
std        764.702082
min        102.000000
25%       1604.000000
50%       1809.000000
75%       2159.000000
max       5158.000000
Name: officer_id, dtype: float64

In [19]:
df.officer_gender.unique()

[M, F]
Categories (2, object): [M, F]

In [20]:
#There are NaNs, but there are unknowns, these will stay unchanged in the dataset
df.officer_race.value_counts(ascending = False, dropna = False)

White                            26248
Hispanic or Latino                1735
Two or More Races                 1588
Black or African American         1342
Asian                             1341
Not Specified                      845
Nat Hawaiian/Oth Pac Islander      318
American Indian/Alaska Native      264
Unknown                              2
Name: officer_race, dtype: int64

In [21]:
df[df['officer_race'] == 'Unknown']

,subject_age,subject_id,stop_resolution,weapon_type,officer_id,officer_age,officer_gender,officer_race,subject_race,subject_gender,date,initial_call_type,call_type,officer_squad,arrest,frisk,precinct,sector,beat
13504,56 and Above,6864.0,GO Report,None,4766,37.0,M,Unknown,Hispanic,Male,2018-07-10,UNKNOWN - ANI/ALI - WRLS PHNS (INCL OPEN LINE),"TELEPHONE OTHER, NOT 911",TRAINING - FIELD TRAINING SQUAD,N,Y,North,B,B2
29071,26 - 35,6490.0,Street Check,None,4766,37.0,M,Unknown,White,Female,2018-05-21,NaN,NaN,TRAINING - FIELD TRAINING SQUAD,N,N,NaN,NaN,NaN


In [22]:
df[df['officer_race'] == 'Not Specified'].head()

,subject_age,subject_id,stop_resolution,weapon_type,officer_id,officer_age,officer_gender,officer_race,subject_race,subject_gender,date,initial_call_type,call_type,officer_squad,arrest,frisk,precinct,sector,beat
68,18 - 25,1806.0,NaN,None,1824,27.0,M,Not Specified,Unknown,Male,2015-07-18,NaN,NaN,WEST PCT 1ST W - KING/QUEEN,N,N,NaN,NaN,NaN
224,46 - 55,13405.0,NaN,None,2169,29.0,M,Not Specified,Black,Male,2017-04-22,ASLT - IP/JO - PERSON SHOT OR SHOT AT,"TELEPHONE OTHER, NOT 911",EAST PCT 3RD W - E/G RELIEF,N,Y,East,G,G2
230,36 - 45,23667.0,NaN,None,1047,49.0,M,Not Specified,White,Male,2018-09-20,ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS),911,WEST PCT 2ND W - K/Q RELIEF,N,N,West,K,K1
241,36 - 45,15030.0,NaN,None,2201,40.0,M,Not Specified,Unknown,Male,2019-04-29,NaN,NaN,SOUTHWEST PCT 2ND W - WILLIAM,N,N,NaN,NaN,NaN
346,1 - 17,NaN,Arrest with GO or Supplemental,Handgun,1576,34.0,M,Not Specified,Black,Male,2016-01-05,UNKNOWN - ANI/ALI - LANDLINE (INCLUDES OPEN LINE),911,SOUTH PCT OPS - NIGHT ACT,Y,Y,South,S,S3


In [23]:
#replace the 'unkown' values with 'Not Specified'
df.replace({'officer_race' : {'Unknown':'Not Specified'}}, inplace = True)

In [24]:
# view the races listed in the dataset
df.subject_race.unique()

[Black, White, NaN, Multi-Racial, Asian, Hispanic, Unknown, American Indian / Alaskan Native, Other]
Categories (8, object): [Black, White, Multi-Racial, Asian, Hispanic, Unknown, American Indian / Alaskan Native, Other]

In [25]:
df.subject_race.value_counts(dropna = False)

White                               16781
Black                               10259
Hispanic                             1647
Unknown                              1540
American Indian / Alaskan Native     1074
Asian                                1014
Multi-Racial                          796
NaN                                   426
Other                                 146
-                                       0
Name: subject_race, dtype: int64

In [26]:
len(df[pd.isnull(df.subject_race)])

426

In [27]:
#drop the rows without null subject data
df = df.drop(df[pd.isnull(df.subject_race)].index)


In [28]:
df.date.describe()

count                   33257
unique                   1513
top       2015-10-01 00:00:00
freq                       88
first     2015-03-16 00:00:00
last      2019-05-07 00:00:00
Name: date, dtype: object

In [29]:
df.call_type.unique()

array(['911', 'ONVIEW', nan, 'TELEPHONE OTHER, NOT 911',
       'ALARM CALL (NOT POLICE ALARM)', 'TEXT MESSAGE',
       'SCHEDULED EVENT (RECURRING)'], dtype=object)

In [30]:
df.initial_call_type.value_counts(dropna = False)

NaN                                                11996
SUSPICIOUS PERSON, VEHICLE OR INCIDENT              1857
DISTURBANCE, MISCELLANEOUS/OTHER                    1636
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW          1593
ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)      1367
                                                   ...  
REQUEST TO WATCH                                       1
PARKS VIOLATIONS/CITES (INCLUDES EXCLUSIONS)           1
-ASSIGNED DUTY - STAKEOUT                              1
HAZ - IMMINENT THRT TO PHYS SAFETY (NO HAZ MAT)        1
ALARM - RESIDENTIAL - SILENT/AUD PANIC/DURESS          1
Name: initial_call_type, Length: 156, dtype: int64

In [31]:
df.initial_call_type.nunique()

155

In [32]:
df.call_type.value_counts(dropna = False)

911                              13739
NaN                              11996
ONVIEW                            5198
TELEPHONE OTHER, NOT 911          2166
ALARM CALL (NOT POLICE ALARM)      156
TEXT MESSAGE                         1
SCHEDULED EVENT (RECURRING)          1
Name: call_type, dtype: int64

In [33]:
df.frisk.unique()

[N, NaN, Y]
Categories (2, object): [N, Y]

In [34]:
df.frisk.value_counts(dropna = False)

N      25657
Y       7191
NaN      409
-          0
Name: frisk, dtype: int64

In [35]:
df = df.drop(df[pd.isnull(df.frisk)].index)

In [36]:
df.arrest.unique()

[N, Y]
Categories (2, object): [N, Y]

In [37]:
#replace string categories with values
clean_flags = {'frisk' : {'Y' : 1, 'N' : 0}, 
              'arrest' : {'Y' : 1, 'N' : 0}}

df.replace(clean_flags, inplace = True)

In [38]:
df.shape

(32848, 19)

In [39]:
#change the layout of the columns and save the cleaned file to csv.
df = df[['date','officer_id','officer_age', 'officer_race', 'officer_gender','officer_squad',
               'subject_id','subject_age','subject_race','subject_gender',
              'stop_resolution','weapon_type','initial_call_type','call_type','arrest', 'frisk',
              'precinct', 'sector', 'beat' ]]



#create the path for the new file
terry_data_clean = os.path.join(PROJ_ROOT, 
                         'data', 'processed',
                         'Terry_Stops_Clean.csv')

df.to_csv(terry_data_clean, index = False)